In [ ]:
# 📦 라이브러리 Import 및 환경 설정
import sys
import os
from pathlib import Path
import warnings

# 프로젝트 루트 경로를 Python path에 추가
project_root = Path().absolute()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# 경고 메시지 필터링
warnings.filterwarnings('ignore')

# 데이터 처리 및 분석
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 머신러닝
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

# 프로젝트 모듈
try:
    from src.analytics.rfm_analysis import RFMAnalyzer
    from src.analytics.clv_analysis import CLVAnalyzer
    from src.analytics.cohort_analysis import CohortAnalyzer
    from src.analytics.customer_segmentation import CustomerSegmentation
    from src.utils.logger import get_logger
    print("✅ 고급 분석 모듈 import 성공")
except ImportError as e:
    print(f"⚠️  분석 모듈 import 실패: {e}")
    print("기본 라이브러리로 분석을 진행합니다.")

# 시각화 설정
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# pandas 옵션 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# 로거 설정
logger = get_logger(__name__)

print("🔧 고급 고객 분석 환경 설정 완료!")
print(f"📍 현재 작업 디렉토리: {os.getcwd()}")
print(f"📊 pandas 버전: {pd.__version__}")
print(f"🔢 numpy 버전: {np.__version__}")
print(f"🤖 scikit-learn 사용 가능")


In [ ]:
# 📂 데이터 로드
print("📂 생성된 데이터를 로드합니다...")

# 데이터 파일 경로 확인
data_dir = Path('data/raw')
files_needed = ['customers.csv', 'products.csv', 'orders.csv', 'order_items.csv']

print("📋 필요한 데이터 파일 확인:")
for file in files_needed:
    file_path = data_dir / file
    if file_path.exists():
        file_size = file_path.stat().st_size / 1024  # KB
        print(f"  ✅ {file}: {file_size:.1f} KB")
    else:
        print(f"  ❌ {file}: 파일이 없습니다!")

print()

# 데이터 로드
try:
    customers_df = pd.read_csv(data_dir / 'customers.csv')
    products_df = pd.read_csv(data_dir / 'products.csv')
    orders_df = pd.read_csv(data_dir / 'orders.csv')
    order_items_df = pd.read_csv(data_dir / 'order_items.csv')
    
    print("✅ 모든 데이터 파일 로드 성공!")
    
    # 데이터 크기 확인
    print("\n📊 로드된 데이터 크기:")
    print(f"  👥 고객: {len(customers_df):,}명")
    print(f"  📦 상품: {len(products_df):,}개")
    print(f"  🛒 주문: {len(orders_df):,}건")
    print(f"  📋 주문상품: {len(order_items_df):,}건")
    
except FileNotFoundError as e:
    print(f"❌ 파일 로드 실패: {e}")
    print("먼저 01_data_generation_and_exploration.ipynb를 실행하여 데이터를 생성해주세요.")

# 날짜 컬럼 변환
print("\n🗓️ 날짜 컬럼 변환 중...")
customers_df['signup_date'] = pd.to_datetime(customers_df['signup_date'])
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'])

print("✅ 날짜 변환 완료")

# 기본 데이터 확인
print("\n📋 데이터 미리보기:")
print("\n👥 고객 데이터 (첫 3행):")
print(customers_df.head(3))

print("\n🛒 주문 데이터 (첫 3행):")
print(orders_df.head(3))


In [ ]:
# 🎯 RFM 분석 실행
print("🎯 RFM 분석을 시작합니다...")
print("="*50)

# RFM 분석기 초기화 (기준일: 데이터의 마지막 주문일 + 1일)
reference_date = orders_df['order_date'].max() + timedelta(days=1)
print(f"📅 분석 기준일: {reference_date.strftime('%Y-%m-%d')}")

# RFM 분석기 생성
rfm_analyzer = RFMAnalyzer(reference_date=reference_date)

# 1️⃣ RFM 점수 계산
print("\n1️⃣ RFM 점수 계산 중...")
rfm_data = rfm_analyzer.calculate_rfm_scores(orders_df, customers_df)

# 2️⃣ 고객 세그먼트 할당
print("\n2️⃣ 고객 세그먼트 할당 중...")
rfm_data = rfm_analyzer.assign_segments()

print("\n" + "="*50)
print("🎉 RFM 분석 완료!")

# 결과 확인
print(f"\n📊 분석 결과 요약:")
print(f"  📋 분석된 고객 수: {len(rfm_data):,}명")
print(f"  🛒 구매 고객 수: {len(rfm_data[rfm_data['frequency'] > 0]):,}명")
print(f"  😴 비구매 고객 수: {len(rfm_data[rfm_data['frequency'] == 0]):,}명")

# RFM 기본 통계
print(f"\n📈 RFM 기본 통계:")
rfm_stats = rfm_data[['recency', 'frequency', 'monetary']].describe()
print(rfm_stats)

# 세그먼트별 분포
print(f"\n🎯 세그먼트별 고객 분포:")
segment_counts = rfm_data['segment'].value_counts()
for segment, count in segment_counts.items():
    percentage = count / len(rfm_data) * 100
    print(f"  {segment}: {count:,}명 ({percentage:.1f}%)")

# 샘플 데이터 확인
print(f"\n📋 RFM 데이터 샘플 (상위 5명):")
sample_cols = ['customer_id', 'recency', 'frequency', 'monetary', 'R_score', 'F_score', 'M_score', 'RFM_score', 'segment']
print(rfm_data[sample_cols].head())


In [ ]:
# 📊 RFM 분석 시각화

print("📊 RFM 분석 결과를 시각화합니다...")

# 1️⃣ 세그먼트별 고객 분포 (파이 차트)
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('🎯 RFM 분석 결과 대시보드', fontsize=16, fontweight='bold')

# 세그먼트 분포
segment_counts = rfm_data['segment'].value_counts()
colors = plt.cm.Set3(np.linspace(0, 1, len(segment_counts)))

axes[0, 0].pie(segment_counts.values, labels=segment_counts.index, autopct='%1.1f%%', 
               colors=colors, startangle=90)
axes[0, 0].set_title('🎯 고객 세그먼트 분포')

# 2️⃣ RFM 점수 분포 (히스토그램)
axes[0, 1].hist([rfm_data['R_score'], rfm_data['F_score'], rfm_data['M_score']], 
                bins=5, alpha=0.7, label=['Recency', 'Frequency', 'Monetary'], 
                color=['skyblue', 'lightgreen', 'lightcoral'])
axes[0, 1].set_title('📊 RFM 점수 분포')
axes[0, 1].set_xlabel('점수 (1-5)')
axes[0, 1].set_ylabel('고객 수')
axes[0, 1].legend()

# 3️⃣ 세그먼트별 평균 RFM 점수
segment_scores = rfm_data.groupby('segment')[['R_score', 'F_score', 'M_score']].mean()
top_segments = segment_scores.head(8)  # 상위 8개 세그먼트만 표시

x = np.arange(len(top_segments))
width = 0.25

axes[1, 0].bar(x - width, top_segments['R_score'], width, label='Recency', color='skyblue', alpha=0.8)
axes[1, 0].bar(x, top_segments['F_score'], width, label='Frequency', color='lightgreen', alpha=0.8)
axes[1, 0].bar(x + width, top_segments['M_score'], width, label='Monetary', color='lightcoral', alpha=0.8)

axes[1, 0].set_title('📈 세그먼트별 평균 RFM 점수')
axes[1, 0].set_xlabel('세그먼트')
axes[1, 0].set_ylabel('평균 점수')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(top_segments.index, rotation=45, ha='right')
axes[1, 0].legend()

# 4️⃣ RFM 평균 점수 분포
axes[1, 1].hist(rfm_data['RFM_avg_score'], bins=20, color='gold', alpha=0.7, edgecolor='black')
axes[1, 1].set_title('📊 RFM 평균 점수 분포')
axes[1, 1].set_xlabel('평균 RFM 점수')
axes[1, 1].set_ylabel('고객 수')
axes[1, 1].axvline(rfm_data['RFM_avg_score'].mean(), color='red', linestyle='--', 
                   label=f'평균: {rfm_data["RFM_avg_score"].mean():.2f}')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

# 📈 핵심 인사이트 출력
print("\n" + "="*60)
print("💡 RFM 분석 핵심 인사이트")
print("="*60)

# 가장 큰 세그먼트
largest_segment = segment_counts.index[0]
largest_pct = segment_counts.iloc[0] / len(rfm_data) * 100
print(f"🎯 가장 큰 세그먼트: {largest_segment} ({largest_pct:.1f}%)")

# 고가치 고객 (Champions + Loyal Customers)
high_value_customers = len(rfm_data[rfm_data['segment'].isin(['Champions', 'Loyal Customers'])])
high_value_pct = high_value_customers / len(rfm_data) * 100
print(f"💎 고가치 고객 비율: {high_value_customers:,}명 ({high_value_pct:.1f}%)")

# 이탈 위험 고객
at_risk_segments = ['At Risk', 'Cannot Lose Them', 'About to Sleep']
at_risk_customers = len(rfm_data[rfm_data['segment'].isin(at_risk_segments)])
at_risk_pct = at_risk_customers / len(rfm_data) * 100
print(f"🚨 이탈 위험 고객: {at_risk_customers:,}명 ({at_risk_pct:.1f}%)")

# 평균 RFM 점수가 높은 고객 (4점 이상)
high_score_customers = len(rfm_data[rfm_data['RFM_avg_score'] >= 4.0])
high_score_pct = high_score_customers / len(rfm_data) * 100
print(f"⭐ 우수 고객 (평균 4점 이상): {high_score_customers:,}명 ({high_score_pct:.1f}%)")

print("\n" + "="*60)


In [ ]:
# 💰 CLV 분석 실행
print("💰 고객 생애 가치 (CLV) 분석을 시작합니다...")
print("="*50)

# CLV 분석기 초기화
clv_analyzer = CLVAnalyzer(analysis_period_months=12)

# 1️⃣ 역사적 CLV 계산
print("\n1️⃣ 역사적 CLV 계산 중...")
clv_data = clv_analyzer.calculate_historical_clv(orders_df, customers_df)

# 2️⃣ 예측 CLV 계산
print("\n2️⃣ 예측 CLV 계산 중...")
clv_data = clv_analyzer.calculate_predictive_clv(orders_df)

# 3️⃣ CLV 기반 고객 세그멘테이션
print("\n3️⃣ CLV 기반 고객 세그멘테이션 중...")
clv_data = clv_analyzer.segment_customers_by_clv()

print("\n" + "="*50)
print("🎉 CLV 분석 완료!")

# 결과 확인
print(f"\n📊 CLV 분석 결과 요약:")
print(f"  📋 분석된 고객 수: {len(clv_data):,}명")

# CLV 기본 통계
clv_stats = clv_data[['total_spent', 'avg_order_value', 'purchase_frequency', 
                      'historical_clv', 'predictive_clv']].describe()
print(f"\n📈 CLV 기본 통계:")
print(clv_stats)

# CLV 세그먼트별 분포
print(f"\n💎 CLV 세그먼트별 분포:")
clv_segment_counts = clv_data['clv_segment'].value_counts()
for segment, count in clv_segment_counts.items():
    percentage = count / len(clv_data) * 100
    avg_clv = clv_data[clv_data['clv_segment'] == segment]['predictive_clv'].mean()
    print(f"  {segment}: {count:,}명 ({percentage:.1f}%) - 평균 CLV: {avg_clv:,.0f}원")

# 상위 CLV 고객 확인
print(f"\n🏆 상위 CLV 고객 TOP 10:")
top_clv_customers = clv_data.nlargest(10, 'predictive_clv')[
    ['customer_id', 'total_spent', 'purchase_frequency', 'historical_clv', 'predictive_clv']
]
print(top_clv_customers)

# 전체 CLV 합계
total_predicted_clv = clv_data['predictive_clv'].sum()
total_historical_spent = clv_data['total_spent'].sum()
print(f"\n💰 전체 예측 CLV: {total_predicted_clv:,.0f}원")
print(f"💳 현재까지 총 매출: {total_historical_spent:,.0f}원")
print(f"📈 예상 추가 수익: {total_predicted_clv - total_historical_spent:,.0f}원")


In [ ]:
# 📊 CLV 분석 시각화

print("📊 CLV 분석 결과를 시각화합니다...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('💰 고객 생애 가치 (CLV) 분석 대시보드', fontsize=16, fontweight='bold')

# 1️⃣ CLV 분포 히스토그램
axes[0, 0].hist(clv_data['predictive_clv'], bins=30, color='gold', alpha=0.7, edgecolor='black')
axes[0, 0].set_title('💰 예측 CLV 분포')
axes[0, 0].set_xlabel('예측 CLV (원)')
axes[0, 0].set_ylabel('고객 수')
axes[0, 0].axvline(clv_data['predictive_clv'].mean(), color='red', linestyle='--',
                   label=f'평균: {clv_data["predictive_clv"].mean():,.0f}원')
axes[0, 0].legend()

# 2️⃣ CLV 세그먼트별 분포
clv_segment_counts = clv_data['clv_segment'].value_counts()
colors = ['gold', 'silver', 'bronze']
axes[0, 1].pie(clv_segment_counts.values, labels=clv_segment_counts.index, 
               autopct='%1.1f%%', colors=colors, startangle=90)
axes[0, 1].set_title('💎 CLV 세그먼트 분포')

# 3️⃣ 역사적 CLV vs 예측 CLV 비교
axes[1, 0].scatter(clv_data['historical_clv'], clv_data['predictive_clv'], 
                   alpha=0.6, c=clv_data['purchase_frequency'], cmap='viridis')
axes[1, 0].plot([0, clv_data['historical_clv'].max()], [0, clv_data['historical_clv'].max()], 
                'r--', alpha=0.8, label='동일선')
axes[1, 0].set_title('📊 역사적 CLV vs 예측 CLV')
axes[1, 0].set_xlabel('역사적 CLV (원)')
axes[1, 0].set_ylabel('예측 CLV (원)')
axes[1, 0].legend()

# 컬러바 추가
cbar = plt.colorbar(axes[1, 0].collections[0], ax=axes[1, 0])
cbar.set_label('구매 빈도')

# 4️⃣ CLV 세그먼트별 특성 비교
segment_stats = clv_data.groupby('clv_segment').agg({
    'predictive_clv': 'mean',
    'purchase_frequency': 'mean',
    'avg_order_value': 'mean'
}).round(0)

x = np.arange(len(segment_stats))
width = 0.25

# 정규화를 위해 최대값으로 나누기
norm_clv = segment_stats['predictive_clv'] / segment_stats['predictive_clv'].max()
norm_freq = segment_stats['purchase_frequency'] / segment_stats['purchase_frequency'].max()
norm_aov = segment_stats['avg_order_value'] / segment_stats['avg_order_value'].max()

axes[1, 1].bar(x - width, norm_clv, width, label='평균 CLV (정규화)', color='gold', alpha=0.8)
axes[1, 1].bar(x, norm_freq, width, label='평균 구매빈도 (정규화)', color='lightblue', alpha=0.8)
axes[1, 1].bar(x + width, norm_aov, width, label='평균 주문금액 (정규화)', color='lightgreen', alpha=0.8)

axes[1, 1].set_title('📈 CLV 세그먼트별 특성 비교 (정규화)')
axes[1, 1].set_xlabel('CLV 세그먼트')
axes[1, 1].set_ylabel('정규화된 값 (0-1)')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(segment_stats.index)
axes[1, 1].legend()

plt.tight_layout()
plt.show()

# 📈 CLV 인사이트 출력
print("\n" + "="*60)
print("💡 CLV 분석 핵심 인사이트")
print("="*60)

# High Value 고객 분석
high_value_customers = clv_data[clv_data['clv_segment'] == 'High Value']
high_value_clv_total = high_value_customers['predictive_clv'].sum()
high_value_pct_customers = len(high_value_customers) / len(clv_data) * 100
high_value_pct_clv = high_value_clv_total / clv_data['predictive_clv'].sum() * 100

print(f"💎 High Value 고객:")
print(f"   - 고객 비율: {len(high_value_customers):,}명 ({high_value_pct_customers:.1f}%)")
print(f"   - CLV 기여도: {high_value_clv_total:,.0f}원 ({high_value_pct_clv:.1f}%)")
print(f"   - 평균 CLV: {high_value_customers['predictive_clv'].mean():,.0f}원")

# 파레토 법칙 확인 (80-20 법칙)
clv_sorted = clv_data.sort_values('predictive_clv', ascending=False)
top_20_pct_customers = int(len(clv_sorted) * 0.2)
top_20_pct_clv = clv_sorted.head(top_20_pct_customers)['predictive_clv'].sum()
top_20_pct_contribution = top_20_pct_clv / clv_data['predictive_clv'].sum() * 100

print(f"\n📊 파레토 법칙 검증:")
print(f"   - 상위 20% 고객: {top_20_pct_customers:,}명")
print(f"   - CLV 기여도: {top_20_pct_clv:,.0f}원 ({top_20_pct_contribution:.1f}%)")

# 성장률 분석
positive_growth = len(clv_data[clv_data['spending_growth_rate'] > 0])
negative_growth = len(clv_data[clv_data['spending_growth_rate'] < 0])
positive_growth_pct = positive_growth / len(clv_data) * 100

print(f"\n📈 고객 성장률 분석:")
print(f"   - 성장 고객: {positive_growth:,}명 ({positive_growth_pct:.1f}%)")
print(f"   - 감소 고객: {negative_growth:,}명 ({100-positive_growth_pct:.1f}%)")

print("\n" + "="*60)


In [ ]:
# 📅 코호트 분석 실행
print("📅 코호트 분석을 시작합니다...")
print("="*50)

# 코호트 분석기 초기화 (월별 분석)
cohort_analyzer = CohortAnalyzer(period_type='monthly')

# 1️⃣ 가입 기반 코호트 생성
print("\n1️⃣ 가입 기반 코호트 생성 중...")
cohort_table = cohort_analyzer.create_signup_cohorts(customers_df, orders_df)

# 2️⃣ 유지율 테이블 생성  
print("\n2️⃣ 유지율 테이블 생성 중...")
retention_table = cohort_analyzer.create_retention_table()

# 3️⃣ 매출 기반 코호트 분석
print("\n3️⃣ 매출 기반 코호트 분석 중...")
revenue_cohort_table = cohort_analyzer.analyze_revenue_cohorts(customers_df, orders_df)

print("\n" + "="*50)
print("🎉 코호트 분석 완료!")

# 결과 확인
print(f"\n📊 코호트 분석 결과 요약:")
print(f"  📅 분석된 코호트 수: {len(retention_table):,}개")
print(f"  📈 최대 추적 기간: {retention_table.shape[1]}개월")

# 유지율 테이블 미리보기
print(f"\n📋 유지율 테이블 (상위 5개 코호트):")
print(retention_table.head().round(3))

# 첫 달 유지율 통계
if 1 in retention_table.columns:
    first_month_retention = retention_table[1].dropna()
    print(f"\n📈 1개월 유지율 통계:")
    print(f"  평균: {first_month_retention.mean():.1%}")
    print(f"  중앙값: {first_month_retention.median():.1%}")
    print(f"  최고: {first_month_retention.max():.1%}")
    print(f"  최저: {first_month_retention.min():.1%}")

# 코호트별 매출 정보
print(f"\n💰 코호트별 매출 정보 (상위 5개):")
revenue_summary = revenue_cohort_table.groupby('cohort_group').agg({
    'total_revenue': 'sum',
    'active_customers': 'mean'
}).round(0).head()
print(revenue_summary)


In [ ]:
# 📊 코호트 분석 시각화
print("📊 코호트 분석 결과를 시각화합니다...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('📅 코호트 분석 대시보드', fontsize=16, fontweight='bold')

# 1️⃣ 유지율 히트맵
import seaborn as sns

# 유지율 테이블을 백분율로 변환
retention_pct = retention_table * 100

sns.heatmap(retention_pct, 
            annot=True, 
            fmt='.1f', 
            cmap='YlOrRd', 
            ax=axes[0, 0],
            cbar_kws={'label': '유지율 (%)'})
axes[0, 0].set_title('🔥 코호트별 유지율 히트맵')
axes[0, 0].set_xlabel('기간 (월)')
axes[0, 0].set_ylabel('가입 코호트')

# 2️⃣ 평균 유지율 트렌드
avg_retention_by_period = retention_table.mean()
axes[0, 1].plot(avg_retention_by_period.index, avg_retention_by_period.values * 100, 
                marker='o', linewidth=2, markersize=6, color='steelblue')
axes[0, 1].set_title('📈 평균 유지율 트렌드')
axes[0, 1].set_xlabel('기간 (월)')
axes[0, 1].set_ylabel('평균 유지율 (%)')
axes[0, 1].grid(True, alpha=0.3)

# 3️⃣ 코호트 크기 분포
cohort_sizes = cohort_table.groupby('cohort_group')['total_customers'].first()
axes[1, 0].bar(range(len(cohort_sizes)), cohort_sizes.values, 
               color='lightgreen', alpha=0.7)
axes[1, 0].set_title('👥 코호트별 고객 수')
axes[1, 0].set_xlabel('코호트 인덱스')
axes[1, 0].set_ylabel('고객 수')

# x축 레이블을 코호트 이름으로 설정 (간소화)
cohort_labels = [str(cohort)[:7] for cohort in cohort_sizes.index]  # YYYY-MM 형태로 단축
axes[1, 0].set_xticks(range(0, len(cohort_sizes), max(1, len(cohort_sizes)//5)))
axes[1, 0].tick_params(axis='x', rotation=45)

# 4️⃣ 코호트별 누적 매출
cumulative_revenue = revenue_cohort_table.groupby('cohort_group')['total_revenue'].sum().sort_index()
axes[1, 1].plot(range(len(cumulative_revenue)), cumulative_revenue.values/1000, 
                marker='s', linewidth=2, markersize=4, color='gold')
axes[1, 1].set_title('💰 코호트별 총 매출')
axes[1, 1].set_xlabel('코호트 인덱스')  
axes[1, 1].set_ylabel('총 매출 (천원)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 📈 코호트 인사이트 출력
cohort_insights = cohort_analyzer.get_cohort_insights()

print("\n" + "="*60)
print("💡 코호트 분석 핵심 인사이트")
print("="*60)

# 유지율 트렌드
if 'retention_trends' in cohort_insights:
    trends = cohort_insights['retention_trends']
    print("📈 유지율 트렌드:")
    
    if trends['1_month_retention']:
        print(f"   1개월 유지율: {trends['1_month_retention']:.1%}")
    if trends['3_month_retention']:
        print(f"   3개월 유지율: {trends['3_month_retention']:.1%}")
    if trends['6_month_retention']:
        print(f"   6개월 유지율: {trends['6_month_retention']:.1%}")

# 비즈니스 인사이트
if 'business_insights' in cohort_insights:
    print("\n🎯 비즈니스 인사이트:")
    for insight in cohort_insights['business_insights']:
        print(f"   {insight}")

# 추천 액션
if 'recommendations' in cohort_insights:
    print("\n📋 추천 액션:")
    for i, recommendation in enumerate(cohort_insights['recommendations'][:5], 1):
        print(f"   {i}. {recommendation}")

# 최고/최저 성과 코호트
if len(retention_table) > 0 and 1 in retention_table.columns:
    best_cohort = retention_table[1].idxmax()
    worst_cohort = retention_table[1].idxmin()
    best_retention = retention_table[1].max()
    worst_retention = retention_table[1].min()
    
    print(f"\n🏆 성과 비교:")
    print(f"   최고 성과 코호트: {best_cohort} ({best_retention:.1%})")
    print(f"   최저 성과 코호트: {worst_cohort} ({worst_retention:.1%})")
    print(f"   성과 차이: {(best_retention - worst_retention):.1%}")

print("\n" + "="*60)


In [ ]:
# 🎯 고객 세그멘테이션 실행
print("🎯 고객 세그멘테이션을 시작합니다...")
print("="*50)

# 고객 세그멘테이션 분석기 초기화
segmentation = CustomerSegmentation(random_state=42)

# 1️⃣ RFM 데이터를 사용한 행동적 세그멘테이션
print("\n1️⃣ 행동적 세그멘테이션 (RFM 기반) 중...")
segmented_data = segmentation.behavioral_segmentation(rfm_data)

# 2️⃣ 인구통계학적 세그멘테이션
print("\n2️⃣ 인구통계학적 세그멘테이션 중...")
segmented_data = segmentation.demographic_segmentation(customers_df)

print("\n" + "="*50)
print("🎉 고객 세그멘테이션 완료!")

# 결과 확인
print(f"\n📊 세그멘테이션 결과 요약:")
print(f"  📋 분석된 고객 수: {len(segmented_data):,}명")

# 행동적 세그먼트 분포
if 'behavioral_segment_name' in segmented_data.columns:
    print(f"\n🎯 행동적 세그먼트 분포:")
    behavioral_counts = segmented_data['behavioral_segment_name'].value_counts()
    for segment, count in behavioral_counts.items():
        percentage = count / len(segmented_data) * 100
        print(f"  {segment}: {count:,}명 ({percentage:.1f}%)")

# 연령대 분포
print(f"\n👥 연령대별 분포:")
age_counts = segmented_data['age_group'].value_counts()
for age_group, count in age_counts.items():
    percentage = count / len(segmented_data) * 100
    print(f"  {age_group}: {count:,}명 ({percentage:.1f}%)")

# 성별 분포  
print(f"\n🚻 성별 분포:")
gender_counts = segmented_data['gender'].value_counts()
for gender, count in gender_counts.items():
    gender_name = "남성" if gender == 'M' else "여성"
    percentage = count / len(segmented_data) * 100
    print(f"  {gender_name}: {count:,}명 ({percentage:.1f}%)")

# 지역별 분포
print(f"\n🗺️ 지역별 분포:")
region_counts = segmented_data['region'].value_counts()
for region, count in region_counts.items():
    percentage = count / len(segmented_data) * 100
    print(f"  {region}: {count:,}명 ({percentage:.1f}%)")

# 세그먼트별 특성 분석
if 'behavioral_segment_name' in segmented_data.columns:
    print(f"\n📈 행동적 세그먼트별 특성:")
    segment_profile = segmented_data.groupby('behavioral_segment_name').agg({
        'frequency': 'mean',
        'monetary': 'mean', 
        'recency': 'mean',
        'age': 'mean'
    }).round(1)
    print(segment_profile)


In [ ]:
# 📊 고객 세그멘테이션 시각화
print("📊 고객 세그멘테이션 결과를 시각화합니다...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('🎯 고객 세그멘테이션 분석 대시보드', fontsize=16, fontweight='bold')

# 1️⃣ 행동적 세그먼트 분포
if 'behavioral_segment_name' in segmented_data.columns:
    behavioral_counts = segmented_data['behavioral_segment_name'].value_counts()
    colors = plt.cm.Set3(np.linspace(0, 1, len(behavioral_counts)))
    
    axes[0, 0].pie(behavioral_counts.values, labels=behavioral_counts.index, 
                   autopct='%1.1f%%', colors=colors, startangle=90)
    axes[0, 0].set_title('🎯 행동적 세그먼트 분포')

# 2️⃣ 인구통계 분포 (연령대 × 성별)
gender_age_crosstab = pd.crosstab(segmented_data['age_group'], segmented_data['gender'])
gender_age_crosstab.plot(kind='bar', ax=axes[0, 1], color=['lightblue', 'lightpink'], alpha=0.8)
axes[0, 1].set_title('👥 연령대별 성별 분포')
axes[0, 1].set_xlabel('연령대')
axes[0, 1].set_ylabel('고객 수')
axes[0, 1].legend(['남성', '여성'])
axes[0, 1].tick_params(axis='x', rotation=45)

# 3️⃣ 지역별 평균 구매 금액
region_monetary = segmented_data.groupby('region')['monetary'].mean().sort_values(ascending=True)
axes[1, 0].barh(region_monetary.index, region_monetary.values, color='lightgreen', alpha=0.7)
axes[1, 0].set_title('🗺️ 지역별 평균 구매 금액')
axes[1, 0].set_xlabel('평균 구매 금액 (원)')

# 4️⃣ RFM 특성별 세그먼트 비교 (행동적 세그먼트가 있는 경우)
if 'behavioral_segment_name' in segmented_data.columns:
    # 세그먼트별 평균 RFM 값 계산
    segment_rfm = segmented_data.groupby('behavioral_segment_name')[['R_score', 'F_score', 'M_score']].mean()
    
    # 레이더 차트 스타일로 표현 (간단한 바 차트로 대체)
    segment_names = segment_rfm.index[:4]  # 상위 4개 세그먼트만 표시
    x = np.arange(len(segment_names))
    width = 0.25
    
    axes[1, 1].bar(x - width, segment_rfm.loc[segment_names, 'R_score'], width, 
                   label='Recency', color='skyblue', alpha=0.8)
    axes[1, 1].bar(x, segment_rfm.loc[segment_names, 'F_score'], width, 
                   label='Frequency', color='lightgreen', alpha=0.8)
    axes[1, 1].bar(x + width, segment_rfm.loc[segment_names, 'M_score'], width, 
                   label='Monetary', color='lightcoral', alpha=0.8)
    
    axes[1, 1].set_title('📊 세그먼트별 RFM 점수 비교')
    axes[1, 1].set_xlabel('행동적 세그먼트')
    axes[1, 1].set_ylabel('평균 점수')
    axes[1, 1].set_xticks(x)
    axes[1, 1].set_xticklabels(segment_names, rotation=45, ha='right')
    axes[1, 1].legend()

plt.tight_layout()
plt.show()

# 📈 세그멘테이션 인사이트 출력
segmentation_insights = segmentation.get_segmentation_insights()

print("\n" + "="*60)
print("💡 고객 세그멘테이션 핵심 인사이트")
print("="*60)

# 행동적 세그먼트 인사이트
if 'behavioral' in segmentation_insights:
    print("🎯 행동적 세그먼트 특성:")
    for segment, data in segmentation_insights['behavioral'].items():
        print(f"\n   📊 {segment}:")
        print(f"      고객 수: {data['customer_count']:,}명")
        print(f"      평균 구매빈도: {data['avg_frequency']:.1f}회")
        print(f"      평균 구매금액: {data['avg_monetary']:,.0f}원")
        print(f"      평균 최근성: {data['avg_recency']:.0f}일")

# 인구통계 분포
if 'demographic' in segmentation_insights:
    demo = segmentation_insights['demographic']
    
    print(f"\n👥 인구통계 분포:")
    print(f"   연령대별: {dict(list(demo['age_distribution'].items())[:3])}")
    print(f"   성별: {demo['gender_distribution']}")
    print(f"   지역별: {dict(list(demo['region_distribution'].items())[:3])}")

# 마케팅 전략 추천
print(f"\n📋 세그먼트별 추천 마케팅 전략:")

# VIP 고객 전략
if 'VIP 고객' in segmented_data['behavioral_segment_name'].values:
    vip_count = len(segmented_data[segmented_data['behavioral_segment_name'] == 'VIP 고객'])
    print(f"\n   💎 VIP 고객 ({vip_count:,}명):")
    print(f"      - 프리미엄 서비스 및 개인 맞춤 상담")
    print(f"      - 독점 상품 우선 판매 및 VIP 이벤트")
    print(f"      - 높은 마케팅 예산 투입 (ROI 최고)")

# 충성 고객 전략  
if '충성 고객' in segmented_data['behavioral_segment_name'].values:
    loyal_count = len(segmented_data[segmented_data['behavioral_segment_name'] == '충성 고객'])
    print(f"\n   💝 충성 고객 ({loyal_count:,}명):")
    print(f"      - 로열티 프로그램 및 정기 할인 혜택")
    print(f"      - 브랜드 앰버서더 프로그램 참여 유도")
    print(f"      - 신제품 우선 체험 기회 제공")

# 신규/일반 고객 전략
if '신규/일반' in segmented_data['behavioral_segment_name'].values:
    new_count = len(segmented_data[segmented_data['behavioral_segment_name'] == '신규/일반'])
    print(f"\n   🌱 신규/일반 고객 ({new_count:,}명):")
    print(f"      - 온보딩 프로그램 및 사용법 가이드")
    print(f"      - 첫 구매 할인 및 무료 배송 혜택")
    print(f"      - 점진적 관계 구축 및 브랜드 교육")

print("\n" + "="*60)


In [ ]:
# 📊 종합 분석 및 비즈니스 인사이트
print("📊 종합 분석 결과를 정리합니다...")
print("="*70)

# 1️⃣ 핵심 메트릭 요약
print("📈 핵심 비즈니스 메트릭")
print("-" * 30)

# RFM 메트릭
high_value_rfm = len(rfm_data[rfm_data['segment'].isin(['Champions', 'Loyal Customers'])])
at_risk_rfm = len(rfm_data[rfm_data['segment'].isin(['At Risk', 'Cannot Lose Them', 'About to Sleep'])])

print(f"👥 전체 고객 수: {len(customers_df):,}명")
print(f"🛒 구매 고객 수: {len(rfm_data[rfm_data['frequency'] > 0]):,}명")
print(f"💎 고가치 고객 (RFM): {high_value_rfm:,}명 ({high_value_rfm/len(rfm_data)*100:.1f}%)")
print(f"🚨 이탈 위험 고객: {at_risk_rfm:,}명 ({at_risk_rfm/len(rfm_data)*100:.1f}%)")

# CLV 메트릭
high_clv_customers = len(clv_data[clv_data['clv_segment'] == 'High Value'])
total_predicted_value = clv_data['predictive_clv'].sum()
avg_clv = clv_data['predictive_clv'].mean()

print(f"\n💰 평균 고객 생애 가치: {avg_clv:,.0f}원")
print(f"🎯 High CLV 고객: {high_clv_customers:,}명 ({high_clv_customers/len(clv_data)*100:.1f}%)")
print(f"💎 전체 예측 CLV: {total_predicted_value:,.0f}원")

# 코호트 메트릭 (1개월 유지율이 있는 경우)
if 1 in retention_table.columns:
    avg_1month_retention = retention_table[1].mean()
    print(f"📅 평균 1개월 유지율: {avg_1month_retention:.1%}")

print("\n" + "="*70)

# 2️⃣ 세그먼트별 상세 분석
print("🎯 고가치 고객 심층 분석")
print("-" * 30)

# RFM Champions + CLV High Value 교집합 분석
champions = rfm_data[rfm_data['segment'] == 'Champions']['customer_id']
high_clv_ids = clv_data[clv_data['clv_segment'] == 'High Value']['customer_id']

# 두 조건을 모두 만족하는 고객 (최우선 고객)
vip_customers = set(champions) & set(high_clv_ids)
print(f"🏆 VIP 고객 (RFM Champions + High CLV): {len(vip_customers):,}명")

if len(vip_customers) > 0:
    # VIP 고객의 특성 분석
    vip_data = clv_data[clv_data['customer_id'].isin(vip_customers)]
    vip_avg_clv = vip_data['predictive_clv'].mean()
    vip_avg_frequency = vip_data['purchase_frequency'].mean()
    vip_total_contribution = vip_data['predictive_clv'].sum() / clv_data['predictive_clv'].sum() * 100
    
    print(f"   💰 평균 CLV: {vip_avg_clv:,.0f}원")
    print(f"   🔄 평균 구매 빈도: {vip_avg_frequency:.1f}회")
    print(f"   📈 전체 CLV 기여도: {vip_total_contribution:.1f}%")

# 3️⃣ 이탈 위험 고객 분석
print(f"\n🚨 이탈 위험 고객 분석")
print("-" * 30)

at_risk_segments = ['At Risk', 'Cannot Lose Them', 'About to Sleep', 'Hibernating']
at_risk_data = rfm_data[rfm_data['segment'].isin(at_risk_segments)]

if len(at_risk_data) > 0:
    # 이탈 위험도별 분류
    cannot_lose = len(rfm_data[rfm_data['segment'] == 'Cannot Lose Them'])
    at_risk = len(rfm_data[rfm_data['segment'] == 'At Risk'])
    about_to_sleep = len(rfm_data[rfm_data['segment'] == 'About to Sleep'])
    
    print(f"🚫 놓치면 안 되는 고객: {cannot_lose:,}명 (즉시 액션 필요)")
    print(f"⚠️  위험 고객: {at_risk:,}명 (재참여 캠페인 필요)")
    print(f"😴 이탈 직전 고객: {about_to_sleep:,}명 (특별 프로모션 필요)")
    
    # 이탈 위험 고객의 잠재 손실 계산
    at_risk_clv_data = clv_data[clv_data['customer_id'].isin(at_risk_data['customer_id'])]
    potential_loss = at_risk_clv_data['predictive_clv'].sum()
    print(f"💸 잠재 손실 CLV: {potential_loss:,.0f}원")

# 4️⃣ 성장 잠재력 고객 분석
print(f"\n🌱 성장 잠재력 고객 분석")
print("-" * 30)

potential_segments = ['Potential Loyalists', 'New Customers', 'Promising']
potential_data = rfm_data[rfm_data['segment'].isin(potential_segments)]

if len(potential_data) > 0:
    new_customers = len(rfm_data[rfm_data['segment'] == 'New Customers'])
    potential_loyalists = len(rfm_data[rfm_data['segment'] == 'Potential Loyalists'])
    promising = len(rfm_data[rfm_data['segment'] == 'Promising'])
    
    print(f"🆕 신규 고객: {new_customers:,}명 (온보딩 프로그램 필요)")
    print(f"🌟 잠재 충성 고객: {potential_loyalists:,}명 (육성 프로그램 필요)")
    print(f"📈 유망 고객: {promising:,}명 (관심 유도 필요)")
    
    # 성장 잠재력 고객의 예상 가치 상승
    potential_clv_data = clv_data[clv_data['customer_id'].isin(potential_data['customer_id'])]
    current_clv = potential_clv_data['historical_clv'].sum()
    predicted_clv = potential_clv_data['predictive_clv'].sum()
    growth_potential = predicted_clv - current_clv
    
    print(f"📈 예상 CLV 성장: {growth_potential:,.0f}원")

print("\n" + "="*70)


In [ ]:
# 📋 액션 플랜 및 다음 단계
print("📋 실행 가능한 액션 플랜")
print("="*50)

print("🏆 1. VIP 고객 관리 (최우선)")
print("   - 개인 맞춤 서비스 제공")
print("   - 프리미엄 제품 우선 판매")
print("   - 전담 고객 관리자 배정")

print("\n🚨 2. 이탈 위험 고객 복구 (긴급)")
print("   - 즉시 재참여 캠페인 실시")
print("   - 특별 할인 및 인센티브 제공")  
print("   - 고객 만족도 조사 및 개선")

print("\n🌱 3. 성장 잠재력 고객 육성 (중장기)")
print("   - 온보딩 프로그램 강화")
print("   - 단계별 혜택 제공")
print("   - 브랜드 교육 및 커뮤니티 구축")

print("\n📊 4. 데이터 모니터링 (지속)")
print("   - 월별 RFM 세그먼트 변화 추적")
print("   - 코호트별 유지율 모니터링")
print("   - CLV 예측 모델 업데이트")

print("\n" + "="*50)
print("🎉 고급 고객 분석 완료!")
print("\n📚 다음 학습 단계:")
print("  3단계: 매출 분석 & 예측 모델링")
print("  4단계: 머신러닝 & 추천 시스템")
print("  5단계: 웹 대시보드 & 배포")
print("\n💡 이 분석 결과를 기반으로 실제 마케팅 전략을 수립해보세요!")
